In [1]:
GPU_NUMBER = 4
DEVICE = 'cpu' if GPU_NUMBER < 0 else 'cuda:{}'.format(GPU_NUMBER)

In [2]:
import sys
sys.path.append("../")

In [3]:
from semanticsegmentation.evaluation import *
import mmcv

In [4]:
cfg = mmcv.Config.fromfile("../experiments/config_standfordbackground_v1.0.0.py")

In [5]:
evaluate_dataset(
    checkpoint="../work_dirs/debug_v1.0.0/iter_100.pth",
    device=DEVICE,
    config=cfg,
)

100%|██████████| 143/143 [00:10<00:00, 13.31it/s]


(0.8418758636663258,
 array([0.93028468, 0.85909922, 0.93095402, 0.90719162, 0.        ,
        0.89930819, 0.        , 0.79037823]),
 array([0.88038866, 0.70807626, 0.79698873, 0.71641071, 0.        ,
        0.76349607, 0.        , 0.67240772]),
 0.5672210173289042)

In [6]:
evaluate_dataset(
    checkpoint="../work_dirs/debug_v1.0.0/iter_95.pth",
    device=DEVICE,
    config=cfg,
)

100%|██████████| 143/143 [00:10<00:00, 14.04it/s]


(0.8412111627342601,
 array([0.93575635, 0.85810069, 0.93179411, 0.89491031, 0.        ,
        0.89850216, 0.        , 0.78680561]),
 array([0.87687366, 0.70582283, 0.79733402, 0.72048527, 0.        ,
        0.76310523, 0.        , 0.66813821]),
 0.5664699025316756)

In [7]:
evaluate_dataset(
    checkpoint="../work_dirs/debug_v1.0.0/iter_5.pth",
    device=DEVICE,
    config=cfg,
)

100%|██████████| 143/143 [00:10<00:00, 14.12it/s]


(0.6294657893856048,
 array([0.82028544, 0.06383544, 0.89974162, 0.00158438, 0.        ,
        0.98301628, 0.        , 0.554597  ]),
 array([0.7648855 , 0.06339896, 0.71682039, 0.00157028, 0.        ,
        0.41817606, 0.        , 0.44778463]),
 0.30157947807206614)

In [8]:
evaluate_dataset(
    checkpoint="../work_dirs/debug_v1.0.0/iter_5.pth",
    device='cpu',
    config=cfg,
)

100%|██████████| 143/143 [00:32<00:00,  4.35it/s]


(0.6294656971812863,
 array([0.82028544, 0.06383474, 0.89974162, 0.00158438, 0.        ,
        0.98301628, 0.        , 0.554597  ]),
 array([0.76488594, 0.06339827, 0.71682039, 0.00157028, 0.        ,
        0.41817589, 0.        , 0.44778463]),
 0.30157942434770424)

## Single image evaluation

In [9]:
import torch, torchvision
import mmseg
import mmcv

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import os
import glob
import json
import numpy as np
from PIL import Image

from semanticsegmentation.class_names import *

from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

from mmseg.datasets import build_dataset, build_dataloader
from mmseg.models import build_segmentor
from mmseg.apis import init_segmentor, inference_segmentor, single_gpu_test, multi_gpu_test

In [10]:
def _evaluate_single_image(
    image_path, 
    checkpoint,
    device,
    config_file='../experiments/config_standfordbackground.py',
    **kwargs
):
    # prepare image data and ground truth
    gt_image_path = image_path.replace("images", "labels").replace(".jpg", ".regions.txt")
    seg_map = np.loadtxt(gt_image_path).astype(np.uint8)
    
    # load config
    cfg = mmcv.Config.fromfile(config_file)
    
    # initiate model
    model = init_segmentor(cfg, checkpoint, device=device)
    
    # assign config to model (Required)
    model.cfg = cfg
    
    # inference
    img = mmcv.imread(image_path)
    result = inference_segmentor(model, img)
    
    return mmseg.core.evaluation.mean_iou(
        results=[result[0]], 
        gt_seg_maps=[seg_map], 
        num_classes=cfg.num_classes, 
        ignore_index=kwargs.get("ignore_index", 255),
        label_map=kwargs.get("label_map", {})
    )

In [11]:
_evaluate_single_image(
    image_path="/rd-temp/mohan/iccv09Data/images/0011003.jpg",
    checkpoint="../work_dirs/debug_v1.0.0/iter_100.pth",
    device=DEVICE
)

/data-non-pii-share/rd_ai/mohan/anaconda3/envs/open-mmlab-2/lib/python3.7/site-packages/mmseg/core/evaluation/metrics.py:215: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label
/data-non-pii-share/rd_ai/mohan/anaconda3/envs/open-mmlab-2/lib/python3.7/site-packages/mmseg/core/evaluation/metrics.py:219: RuntimeWarning: invalid value encountered in true_divide
  iou = total_area_intersect / total_area_union


(0.9595803990610329,
 array([0.97981962, 0.        , 0.92843427, 0.9930413 ,        nan,
               nan,        nan, 0.4151671 ]),
 array([0.96882432, 0.        , 0.92816007, 0.91442277,        nan,
        0.        ,        nan, 0.37319469]))